# Baby Predictions

Sister is pregnant, thought it would be interesting to try to predict due date and the baby's weight.

Found the CDC public dataset [National Center for Health Statistics / Birth Data](https://www.cdc.gov/nchs/data_access/VitalStatsOnline.htm#Births):
* Trusted agency supplying reliable data with clear documentation and transparency providing confidence
* Already anonymised meaning I didn't have to worry about exposing sensistive information, while at the person level there is no identifying information
* Data is available for research and analytical use, see [Data User Agreement](https://www.cdc.gov/nchs/policy/data-user-agreement.html)

Independent Variables

|Source Name|Feature|Rationale|
|-|-|-|
|MAGER|mother_age_yrs|Younger mothers may be more likely to have a shorter gestation|
|TBO_REC| previous_pregnancies_cnt |First time pregnancies may be more likely to have a shorter gestation|
|ILLB_R|interval_since_last_live_birth_mths||
|CIG_0|cigarettes_pre_daily_cnt|Smoking is know to affect fetal development, could cause shorter gestation and lighter birth weight|
|CIG_1|cigarettes_1st_daily_cnt||
|CIG_2|cigarettes_2nd_daily_cnt||
|CIG_3|cigarettes_3rd_daily_cnt||
|CIG_REC|is_smoker||
|BMI|mother_pre_bmi|Mothers size may affect the size of the baby|
|RF_PPTERM|previous_pre_term_birth|Mothers with previous premature births may be more likely to further shorter pregnancies and therefore smaller babies|

## Set-up

In [ ]:
# import standard packages
import json
from matplotlib import colors
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
width = 800
height = 500

## Load data

In [ ]:
# data stored in fixed width, loading schema defining where to split data
with open("../references/schema.json") as f:
    schema = json.load(f)
    f.close()

# definition lists position according to documentation but needs
# to be corrected to half-intervals for pandas
for k, v in schema.items():
    v = list(v.split(","))
    v[0] = int(v[0]) - 1
    v[1] = int(v[1])
    schema[k] = tuple(v)

In [ ]:
# define the features of interest
with open("../references/features.json") as f:
    features = json.load(f)
    f.close()

In [ ]:
# if you have and want to use the full source file uncommment these variables and pd.read_fwf nrows kwarg
# path = "../data/raw/Nat2024PublicUS.c20250512.r20250708.txt"
# sample_size = 100000

# using the already sampled file
path = "../data/interim/Nat2024PublicUS.c20250512.r20250708_sample.txt"

# read in the data
df = pd.read_fwf(
    path,
    colspecs = list(schema.values()),
    # nrows = sample_size,
    header = None,
    names = list(schema.keys()),
    usecols = list(features.keys())
)

# rename fields for readability
df.rename(
    columns = features,
    inplace = True
)

## Cleaning & Preparing

In [ ]:
# split into train and test sets
df_train, df_test = train_test_split(
    df,
    train_size = 0.8,
    random_state = 42
)

In [ ]:
# what have we got overall?
df_train.info(verbose = True, show_counts = True)

In [ ]:
df_train.head()

No missing data and a mix of categorial and numeric data as expected from user guide.

### Unknown

According the guidance document columns where the value is unknown by definition will be encoded with 99-9999 or "U" depending on the scale and data type of the column.

To determine how to treat let's first evaluate how prevalent this is.

In [ ]:
# how many rows have unknown values?
display(
    df_train
    .melt()
    .query("`value` == 99 or `value` == 99.9 or `value` == 999 or `value` == 9999 or `value` == 'U'")
    .groupby("variable")
    .agg(
        unknown_cnt = ("value", "count")
    )
    .assign(
        records_cnt = len(df_train),
        unknown_pct = lambda x: x["unknown_cnt"] / x["records_cnt"]
    )
    .style.format({
        "unknown_cnt" : "{:,.0f}",
        "records_cnt" : "{:,.0f}",
        "unknown_pct" : "{:,.1%}"
    })
)

By column unknowns are not a significant proportion and overall I'd still expect a fraction to be affected. Well above the minimum sample size needed, let's exclude these records.

In [ ]:
def func_remove_unknowns(dataframe: pd.DataFrame):
    """Removes rows with at least one unknown value in any column.
    Returns:
        Dataframe
    """
    mask = (
        (dataframe[dataframe.columns] != 99).all(axis = 1)
        & (dataframe[dataframe.columns] != 99.9).all(axis = 1)
        & (dataframe[dataframe.columns] != 999).all(axis = 1)
        & (dataframe[dataframe.columns] != 9999).all(axis = 1)
        & (dataframe[dataframe.columns] != "U").all(axis = 1)
    )

    return dataframe.loc[mask, :]

In [ ]:
df_train = func_remove_unknowns(df_train)
df_train.info()

### Categorial Encoding

In [ ]:
# categorical data
def func_categorical_to_dummies(dataframe: pd.DataFrame):
    """Encodes all categorical columns (based on data type).
    Drops original columns.
    Returns:
        Dataframe
    """
    dataframe = pd.get_dummies(
        data = dataframe,
        columns = dataframe.select_dtypes(object).columns,
        drop_first = True
    )
    return dataframe

df_train = func_categorical_to_dummies(df_train)

df_train.head()

In [ ]:
# what are the summary statistics?
df_train.describe().transpose()

`interval_since_last_live_birth_mths` has a maximum value of 888 present and a minimum of 3.

Checking the user guide definitions
* 0-3 - 'Plural delivery'
* 888 - 'Not applicable / 1st natality event'

Need to decide how to treat both these scenarios.

### Previous Pregnancies, Birth Intervals and First Birth

In [ ]:
# how many records have an unknown birth interval?
display(
    df_train
    .assign(
        is_na = lambda x: (x["interval_since_last_live_birth_mths"] == 888)
    )
    .pivot_table(
        index = "previous_pregnancies_cnt",
        columns = "is_na",
        values = "mother_age_yrs",
        aggfunc = "count",
        dropna = False,
        fill_value = 0,
        margins = True
    )
)

Majority of not applicable intervals are concentrated with a `previous_pregnancies_cnt` of 1. Checking user guide if the record is the mother's first birth it is included in the count. This makes sense as if it is a mother's first pregnancy, there is no previous birth to count against.

888 is not a meaningful value and linear regression requires non null values. Instead let's replace these values with the mother's age in months - effectively their age is the interval since their last birth.

Additionally, given the distribution it may be useful to add a feature designating whether the record is a first birth or not.

In [ ]:
# how many plural births?
display(
    df_train
    .assign(
        is_multiple = lambda x: (x["interval_since_last_live_birth_mths"] <= 3)
    )
    .pivot_table(
        index = "is_multiple",
        values = "mother_age_yrs",
        aggfunc = "count",
        dropna = False,
        fill_value = 0,
        margins = True
    )
    .style.format("{:0,.0f}")
)

Majority of births are single and in addition the scope of the project is for single births only, as such will exclude these records.

In [ ]:
def func_add_is_first_birth(dataframe: pd.DataFrame):
    """Adds a column flagging whether the record is a first birth, based on the number of previous pregnancies."""
    dataframe["is_first_birth"] =  dataframe["previous_pregnancies_cnt"] == 1
    return dataframe

In [ ]:
df_train = func_add_is_first_birth(df_train)

In [ ]:
def func_remove_na_intervals(dataframe: pd.DataFrame):
    """Excludes records which have an unknown birth interval or plural births but are not the first birth."""
    mask = (
        (dataframe["previous_pregnancies_cnt"] != 1)
        & (
            (dataframe["interval_since_last_live_birth_mths"] == 888)
            | (dataframe["interval_since_last_live_birth_mths"] <= 3)
        )
    )
    dataframe = dataframe.loc[~mask, :]
    return dataframe

In [ ]:
df_train = func_remove_na_intervals(df_train)

In [ ]:
def func_replace_first_birth_interval(dataframe: pd.DataFrame):
    """For first births replaces n/a value with mothers age in months."""
    conditions = [
        dataframe["interval_since_last_live_birth_mths"] == 888
    ]

    values = [
        dataframe["mother_age_yrs"] * 12
    ]

    dataframe["interval_since_last_live_birth_mths"] = np.select(
        conditions,
        values,
        default = dataframe["interval_since_last_live_birth_mths"]
    )
    return dataframe

In [ ]:
df_train = func_replace_first_birth_interval(df_train)

In [ ]:
df_train.info()

### Distributions

In [ ]:
def func_basic_histogram(feature):
    fig = px.histogram(
        df_train,
        x = feature,
        marginal = "box"
    )

    fig.update_layout(
        title = f"Distribution of {feature}",
        width = width,
        height = height
    )

    fig.show()

In [ ]:
# plot each features distribution
for c in df_train.columns:
    func_basic_histogram(c)

Observations:
* `mother_age_yrs` - normally distributed and looks to be few outliers. Trust the accuracy of measurement so will not modify.
* `previous_pregnancies_cnt` - decaying distributions, will log transform to support the linear regression model.
* `interval_since_last_live_birth_mths` - complex distribution with two elements; a Possion form in the low range and a normal distribution in the mid to high range, likely from where the values were replaced by the mother's age. Let's see if a random forest model can determine a pattern, will not modify.
* `cigarettes_pre/1st/2nd/3rd_daily_cnt` - concentrated on zero. Possibly because attitudes to smoking have changed, behaviour change when a mother realises their pregnant or under-reporting due to social perception. Highly biased and with numeric values suspect a model will struggle to differentiate. Will remove columns because of this and rely on `is_smoker_Y` which is slightly less biased.
* `mother_pre_bmi` - mostly normal distribution with a long tail of high BMI values. Suspect this is because the dataset is sourced entirely from America which is known to have some of the highest obesity rates in the world. As trying to predict generally will exclude upper limit outliers.
* `combined_gestation_wks` - normal distribution and depedendent variable, will not modify.
* `baby_weight_g` - normal distribution and depedendent variable, will not modify. Note there are some peaks around specific values, expect this is a measurement error due to rounding to convenient or easy to remember values.
* `is_smoker_Y` - based to False but lets see if random forest can determine, will not modify.
* `previous_pre_term_birth_Y` - based to false but lets see if random forest can determine, will not modify.
* `previs_first_birth` - slight bias to false, will not modify.

In [ ]:
def func_remove_columns(dataframe):
    """Remove unneccessary cigarette consumption columns."""
    dataframe.drop(
        columns = ["cigarettes_pre_daily_cnt", "cigarettes_1st_daily_cnt", "cigarettes_2nd_daily_cnt", "cigarettes_3rd_daily_cnt"],
        inplace = True 
    )
    return dataframe

In [ ]:
df_train = func_remove_columns(df_train)

In [ ]:
def func_log_transform(dataframe):
    """Log transform previous pregnancies"""
    dataframe["previous_pregnancies_cnt"] = np.log(dataframe["previous_pregnancies_cnt"])
    return dataframe

In [ ]:
df_train = func_log_transform(df_train)

In [ ]:
# treat outliers
# dictionary to store limits per column for re-use
outlier_limits = {}

# define columns to define limits
outlier_features = ["mother_pre_bmi"]

# define limits for each column and store
for outlier in outlier_features:
    q1 = df_train[outlier].quantile(0.25)
    q3 = df_train[outlier].quantile(0.75)
    iqr = q3 - q1
    upper_limit = q3 + (1.5 * iqr)
    lower_limit = q1 - (1.5 * iqr)
    outlier_limits.update({
        outlier : {
            "upper_limit" : upper_limit,
            "lower_limit" : lower_limit
        }
    })

In [ ]:
def func_exclude_outliers(outlier_limits, dataframe):
    """Exclude outlier values"""
    for k, v in outlier_limits.items():
        mask = (
            dataframe[k].between(v["lower_limit"], v["upper_limit"], "both")
        )
        dataframe = dataframe.loc[mask, :]
    return dataframe

## Analysis

In [ ]:
# final summary statistics
df_train.describe(include=[int, float, bool]).round(2).transpose()

In training data mean baby weight is 3.25kg with a standard deviation of 0.55kg - can the prediction model beat the standard deviation for accuracy?

### Numeric values

In [ ]:
# using a pearson test of correlation of numeric
rho = df_train.select_dtypes([int, float]).corr(method = "pearson")
pval = df_train.select_dtypes([int, float]).corr(method = lambda x, y: stats.pearsonr(x, y)[1])

In [ ]:
display(
    pval
    .style
    .format("{:.2f}")
    .background_gradient(cmap='Reds')
    .set_table_styles([
        {"selector": "th", "props" :[('max-width', '100px'), ("word-break", "break-all")]},
        {"selector": "td", "props": "text-align : center"},
    ])
)

In [ ]:
cm = sns.diverging_palette(250, 250, as_cmap=True)
divnorm = colors.TwoSlopeNorm(vmin=rho.values.min(), vcenter=0, vmax=rho.values.max())

display(
    rho
    .style
    .format("{:.3f}")
    .background_gradient(cmap=cm, axis = None, gmap=rho.apply(divnorm))
    .set_table_styles([
        {"selector": "th", "props" :[('max-width', '100px'), ("word-break", "break-all")]},
        {"selector": "td", "props": "text-align : center"},
    ])
)

All correlations coefficients are statsically significant and not due to random chance.

For `combined_gestation_wks` all the correlations with the independent variables are very weak; the largest has a vlaue of -0.067.

For `baby_weight_g` the strongest correlation is with `combined_gestation_wks`. This makes sense as the longer the baby has gestating the more weight it can gain.

### Boolean

In [ ]:
# using a spearman test of correlation
columns = df_train.select_dtypes([bool]).columns.to_list() + ["combined_gestation_wks", "baby_weight_g"]

rho = df_train[columns].corr(method = "spearman")
pval = df_train[columns].corr(method = lambda x, y: stats.spearmanr(x, y)[1])

In [ ]:
display(
    pval
    .style
    .format("{:.2f}")
    .background_gradient(cmap='Reds')
    .set_table_styles([
        {"selector": "th", "props" :[('max-width', '100px'), ("word-break", "break-all")]},
        {"selector": "td", "props": "text-align : center"},
    ])
)

In [ ]:
cm = sns.diverging_palette(250, 250, as_cmap=True)
divnorm = colors.TwoSlopeNorm(vmin=rho.values.min(), vcenter=0, vmax=rho.values.max())

display(
    rho
    .style
    .format("{:.3f}")
    .background_gradient(cmap=cm, axis = None, gmap=rho.apply(divnorm))
    .set_table_styles([
        {"selector": "th", "props" :[('max-width', '100px'), ("word-break", "break-all")]},
        {"selector": "td", "props": "text-align : center"},
    ])
)

Again very low correlation coefficients for `combined_gestation_wks` and `baby_weight_g`.

In [ ]:
fig = px.scatter(
    df_train,
    x = "combined_gestation_wks",
    y = "baby_weight_g"
)

fig.update_layout(
    title = "How does gestation length and baby weight relate?",
    width = width,
    height = height
)

fig.show()

## Birth Weight Prediction

Given the very low correlation coefficients between the identified independent and dependent variables the chosen approach is use only `combination_gestation_wks` as an independent variable for the linear regressor and a random forest regressor. A random forest regressor will be used with all the original indepedent variables in case it can idenitfy more complex patterns.

Models to compare:
1. Linear - `combination_gestation_wks` only
2. Random forest - all independent variables
3. Random forest - `combination_gestation_wks` only

In [ ]:
# apply transformation steps to test data
# this model will only be able to predict for known measurements
df_test = func_remove_unknowns(df_test)
df_test = func_remove_na_intervals(df_test)

# feature additions and transformations
# outliers will not be removed as this is the test data
df_test = func_categorical_to_dummies(df_test)
df_test = func_add_is_first_birth(df_test)
df_test = func_replace_first_birth_interval(df_test)
df_test = func_remove_columns(df_test)
df_test = func_log_transform(df_test)

### Linear Model

In [ ]:
independent_vars = ["combined_gestation_wks"]
dependent_var = "baby_weight_g"

ln_regr = LinearRegression()
ln_regr.fit(df_train[independent_vars], df_train[dependent_var])

In [ ]:
print(f"Coefficient is: {ln_regr.coef_[0] : 0,.3f}\nIntercept is: {ln_regr.intercept_ : 0,.3f}")

In [ ]:
# generate a prediction
y_pred = ln_regr.predict(df_test[independent_vars])

In [ ]:
# evaluating model
rmse = root_mean_squared_error(
    y_true = df_test["baby_weight_g"],
    y_pred = y_pred
)


r_squared = r2_score(
    y_true = df_test["baby_weight_g"],
    y_pred = y_pred
)

print(f"Liner regression error metrics\nRoot mean squared error of {rmse:.3f}\nR-squared of: {r_squared: .3f}")

### Random Forest Model

#### All independent

In [ ]:
independent_vars = [
    "combined_gestation_wks",
    "interval_since_last_live_birth_mths",
    "is_first_birth",
    "is_smoker_Y",
    "mother_age_yrs",
    "mother_pre_bmi",
    "previous_pregnancies_cnt",
    "previous_pre_term_birth_Y"
]
dependent_var = "baby_weight_g"

rf_rgr_all = RandomForestRegressor()
rf_rgr_all.fit(
    X = df_train[independent_vars],
    y = df_train[dependent_var]
)

In [ ]:
y_pred = rf_rgr_all.predict(df_test[independent_vars])

In [ ]:
# evaluating model
rmse = root_mean_squared_error(
    y_true = df_test["baby_weight_g"],
    y_pred = y_pred
)


r_squared = r2_score(
    y_true = df_test["baby_weight_g"],
    y_pred = y_pred
)

print(f"Random Forest error metrics\nRoot mean squared error of {rmse:.3f}\nR-squared of: {r_squared: .3f}")

#### Single independent variable

In [ ]:
independent_vars = ["combined_gestation_wks"]
dependent_var = "baby_weight_g"

rf_rgr_one = RandomForestRegressor()
rf_rgr_one.fit(
    X = df_train[independent_vars],
    y = df_train[dependent_var]
)

In [ ]:
y_pred = rf_rgr_one.predict(df_test[independent_vars])

In [ ]:
# evaluating model
rmse = root_mean_squared_error(
    y_true = df_test["baby_weight_g"],
    y_pred = y_pred
)


r_squared = r2_score(
    y_true = df_test["baby_weight_g"],
    y_pred = y_pred
)

print(f"Random Forest error metrics\nRoot mean squared error of {rmse:.3f}\nR-squared of: {r_squared: .3f}")

### Comparison

Summarising the model scores:

|Model|Independent Variables|RMSE| R<sup>2</sup> |
|-|-|-|-|
|Linear|1|489.91|0.21|
|Random Forest|8|501.91|0.17|
|Random Forest|1|476.05|0.25|

The random forest model just using `combined_gestation_wks` produced the best predictions. However, the fit is still much lower than desired. Likely this is due to the variation in the data and the accuracy of measurements. For example, gestation period is recorded at the week level not the day level.

## Final Prediction

In [ ]:
# generate a prediction using the single variable random forest
pred_data = pd.DataFrame(
    data = [df_test["combined_gestation_wks"].median()],
    columns = ["combined_gestation_wks"]
)

prediction = rf_rgr_one.predict(pred_data)
print(f"Predicted baby weight: {prediction[0]:0,.0f}g")

In [ ]:
print(f"Error in prediction as a percentage: {rmse / prediction[0]: 0,.1%}")

In [ ]:
print(f"Predictions based on test data standard statistics\nMedian weight: {df_test["baby_weight_g"].median(): 0,.0f}g\nMean weight: {df_test["baby_weight_g"].mean():0,.0f}g")

In [ ]:
fig = px.scatter(
    df_test,
    x = "combined_gestation_wks",
    y = "baby_weight_g"
)

pred_data = pd.DataFrame(
    data = range(20,48, 1),
    columns = ["combined_gestation_wks"]
)

prediction = rf_rgr_one.predict(pred_data)

fig.add_trace(go.Scatter(
    x = pred_data["combined_gestation_wks"],
    y = prediction,
    name = "prediction"
))

fig.update_layout(
    title = "Prediction versus training data",
    width = width,
    height = height
)

fig.show()

## Conclusion

Gestation duration, and therefore due date, can't be predicted from the independent variables identified. Overall it's concluded that these factors don't impact the gestation period.

Gestation duration can be used to predicted birth weight. There is a high spread in weights but the model was able to predict a result with an RMSE less than the standard deviation. From the plot we can see that, outside the extreme low weight values, the random forest model followed the positive correlation until around the median gestation when weight appears to drop slightly.

Next steps could include:
* Exploring other features in the dataset for influcence e.g. use of infertility treatments
* Finding new datasets that include more detailed measurements e.g. fetus length or head size at key intervals